<a href="https://colab.research.google.com/github/Sonkevin8/SnapCal-Main-Resnet-Retrained/blob/main/retrain_SnapCalMain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#MAIN

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data
from collections import deque
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.io import decode_image
from tqdm.notebook import tqdm
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from sklearn.metrics import mean_absolute_error
from torch.utils.data import DataLoader, default_collate, Dataset, random_split
import torch.optim as optim
import timm
import matplotlib.pyplot as plt
import numpy as np
import sys
import pickle
import h5py
from PIL import Image
from PIL import UnidentifiedImageError
from sklearn.model_selection import train_test_split
from PIL import Image, UnidentifiedImageError
import cv2
from tqdm import tqdm

from google.colab import auth, drive

auth.authenticate_user()
drive.mount('/content/drive', force_remount= True)

img_folder = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/images/"
csv_file = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/labels.csv"

print('Pytorch version: ', torch.__version__)
print('Pandas Version: ' , pd.__version__)
print('Numpy Version: ' , np.__version__)
print('Timm Version: ' , timm.__version__)

df = pd.read_csv(csv_file)
print(df.head())
print(df.columns)

os.listdir('/content')

Mounted at /content/drive
Pytorch version:  2.8.0+cu126
Pandas Version:  2.2.2
Numpy Version:  2.0.2
Timm Version:  1.0.22
   id  original_dish_id   calories
0 NaN        1556572657  41.399998
1 NaN        1556573514   6.440000
2 NaN        1556575014  71.299995
3 NaN        1556575083  27.520000
4 NaN        1556575124   4.480000
Index(['id', 'original_dish_id', 'calories'], dtype='object')


['.config', 'drive', 'sample_data']

In [ ]:

import os
import pandas as pd

csv_file = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/labels.csv"
img_folder = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/images"

# --- Load CSV ---
labels_df = pd.read_csv(csv_file)
print(f"📥 Loaded {len(labels_df)} total samples")

# --- Remove corrupted / invalid entries ---
labels_df = labels_df[labels_df["original_dish_id"] != 1564159636]

# --- Filter by calorie range (0 < cal <= 800) ---
labels_df = labels_df[(labels_df["calories"] > 0) & (labels_df["calories"] <= 800)].reset_index(drop=True)
print(f"Filtered valid calorie range (0–800 kcal) | Remaining: {len(labels_df)} samples")

# --- Clean dish IDs to match image filenames ---
labels_df["original_dish_id"] = (
    labels_df["original_dish_id"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.strip()
)

# --- Verify RGB + Gray image existence ---
rgb_files = {f.split("_")[0] for f in os.listdir(img_folder) if "_rgb.png" in f}
gray_files = {f.split("_")[0] for f in os.listdir(img_folder) if "_gray.png" in f}

valid_ids = sorted(list(rgb_files & gray_files))
labels_df = labels_df[labels_df["original_dish_id"].isin(valid_ids)].reset_index(drop=True)

print(f"Matched {len(labels_df)} valid image pairs (RGB + Gray)")
print("Removed any IDs missing required modalities or exceeding calorie limits.")

📥 Loaded 3493 total samples
Filtered valid calorie range (0–800 kcal) | Remaining: 3197 samples
Matched 3194 valid image pairs (RGB + Gray)
Removed any IDs missing required modalities or exceeding calorie limits.


In [ ]:
#simplified resnet model for RGB images only
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ResNetRGB(nn.Module):
    def __init__(self):
        super(ResNetRGB, self).__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(base.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        feat = self.pool(self.backbone(x)).flatten(1)
        return self.fc(feat)

model_rgb = ResNetRGB().to(device)
criterion = nn.SmoothL1Loss(beta=10.0)
optimizer = optim.Adam(model_rgb.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)


In [ ]:
import os
import pandas as pd
from PIL import Image
from PIL import UnidentifiedImageError
import cv2
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =====================================
# === Image loader with fallback=======
# =====================================

def safe_open(img_dir, dish_id, suffix, mode):
    for ext in ['.png', '.PNG', '.jpg', '.JPG']:
        path = os.path.join(img_dir, f"{dish_id}_{suffix}{ext}")
        if os.path.exists(path):
            try:
                img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
                if img is None or img.size == 0:
                    print(f"Skipping unreadable {suffix} image: {path}")
                    return Image.new(mode, (448, 448))
                if img.dtype != np.uint8:
                    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                if len(img.shape) == 2:
                    pil_img = Image.fromarray(img).convert(mode)
                else:
                    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).convert(mode)
                return pil_img
            except Exception as e:
                print(f"Skipping corrupted {suffix} image: {path} ({e})")
                return Image.new(mode, (448, 448))
    # file not found
    print(f" Missing file for {dish_id}_{suffix}")
    return Image.new(mode, (448, 448))

# =========================
# === Dataset Class =======
# =========================


class MultiModalDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, labels_df, transform_rgb=None, transform_mono=None):
        self.image_dir = image_dir
        self.transform_rgb = transform_rgb
        self.transform_mono = transform_mono
        # --- Clean up IDs to match actual filenames ---
        labels_df['original_dish_id'] = (
            labels_df['original_dish_id']
            .astype(str)
            .str.replace('.0', '', regex=False)
            .str.strip()
        )

        valid_ids = sorted(set(f.split('_')[0] for f in os.listdir(img_folder)))
        labels_df = labels_df[labels_df['original_dish_id'].isin(valid_ids)].reset_index(drop=True)
        # print(f"✅ Filtered to {len(labels_df)} valid samples (removed {3493 - len(labels_df)} missing ones)") # Commented out to avoid reprinting

        self.image_ids = labels_df['original_dish_id'].tolist()
        self.labels = dict(zip(self.image_ids, labels_df['calories'].values))

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        dish_id = self.image_ids[idx]

        # --- Use safe_open for robust image loading ---
        rgb_img = safe_open(self.image_dir, dish_id, "rgb", "RGB")
        # heat_img = safe_open(self.image_dir, dish_id, "gray", "L") # Not needed for RGB-only model

        # --- Apply transforms ---
        if self.transform_rgb:
            rgb_img = self.transform_rgb(rgb_img)
        # if self.transform_mono: # Not needed for RGB-only model
        #     if isinstance(heat_img, Image.Image):
        #       heat_img = self.transform_mono(heat_img)

        cal = torch.tensor(self.labels[dish_id], dtype=torch.float)

        return rgb_img, cal

# --- Paths ---
img_folder = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/images/"
csv_file = "/content/drive/MyDrive/kevin/nutrition5k_cleaned/labels.csv"

# --- Load CSV ---
labels_df = pd.read_csv(csv_file)
# print(f"📥 Loaded {len(labels_df)} total samples")

# --- Remove corrupted / invalid entries ---
labels_df = labels_df[labels_df["original_dish_id"] != 1564159636]

# --- Filter by calorie range (0 < cal <= 800) ---
labels_df = labels_df[(labels_df["calories"] > 0) & (labels_df["calories"] <= 800)].reset_index(drop=True)
# print(f"Filtered valid calorie range (0–800 kcal) | Remaining: {len(labels_df)} samples")

# --- Clean dish IDs to match image filenames ---
labels_df["original_dish_id"] = (
    labels_df["original_dish_id"]
    .astype(str)
    .str.replace(".0", "", regex=False)
    .str.strip()
)

# --- Verify RGB image existence ---
rgb_files = {f.split("_")[0] for f in os.listdir(img_folder) if "_rgb.png" in f}
valid_ids = sorted(list(rgb_files))
labels_df = labels_df[labels_df["original_dish_id"].isin(valid_ids)].reset_index(drop=True)
# print(f"Matched {len(labels_df)} valid RGB images")

# ============================================
# ✂️ TRAIN / VAL / TEST SPLITS
# ============================================
train_df, temp_df = train_test_split(labels_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# ============================================
# 🧠 DATA AUGMENTATION
# ============================================
train_transform_rgb = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(
        brightness=0.25, contrast=0.25,
        saturation=0.2, hue=0.05
    ),
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform_rgb = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ============================================
# 🖼️ DATASET INITIALIZATION
# ============================================
train_dataset = MultiModalDataset(
    image_dir=img_folder,
    labels_df=train_df,
    transform_rgb=train_transform_rgb,
    transform_mono=None, # Only RGB is used in this model
)

val_dataset = MultiModalDataset(
    image_dir=img_folder,
    labels_df=val_df,
    transform_rgb=val_test_transform_rgb,
    transform_mono=None, # Only RGB is used in this model
)

# ============================================
# 🧺 DATALOADERS
# ============================================
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset,   batch_size=8, shuffle=False, num_workers=0)


# --- Define model (moved from previous cell to ensure it's defined) ---
class ResNetRGB(nn.Module):
    def __init__(self):
        super(ResNetRGB, self).__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.backbone = nn.Sequential(*list(base.children())[:-2])
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        feat = self.pool(self.backbone(x)).flatten(1)
        return self.fc(feat)

model_rgb = ResNetRGB().to(device)
criterion = nn.SmoothL1Loss(beta=10.0)
optimizer = optim.Adam(model_rgb.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# Changed checkpoint path for retraining
checkpoint_path = "/content/drive/MyDrive/nutrition_model_checkpoints/best_resnet50_rgb_retrained.pt"

start_epoch = 0
best_val_loss = float('inf')

# Removed checkpoint loading due to persistent KeyError, training from scratch
print(f"❌ Checkpoint could not be loaded. Starting RGB-only ResNet-50 training from scratch. Saving to {os.path.basename(checkpoint_path)}")

num_epochs = 50
best_val_loss = float('inf')

for epoch in range(start_epoch, num_epochs):
    model_rgb.train()
    train_losses, train_mae = [], []

    for rgb, cal in tqdm(train_loader, desc=f"[ResNet50-RGB] Epoch {epoch+1} Training"):
        rgb, cal = rgb.to(device), cal.to(device)
        optimizer.zero_grad()
        preds = model_rgb(rgb).squeeze()
        loss = criterion(preds, cal)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_mae.append(torch.abs(preds - cal).mean().item())

    avg_train_loss = np.mean(train_losses)
    avg_train_mae = np.mean(train_mae)

    # ---- VALIDATION ----
    model_rgb.eval()
    val_losses, val_mae = [], []
    all_preds, all_true = [], []

    with torch.no_grad():
        for rgb, cal in tqdm(val_loader, desc=f"[ResNet50-RGB] Epoch {epoch+1} Validation"):
            rgb, cal = rgb.to(device), cal.to(device)
            preds = model_rgb(rgb).squeeze()
            loss = criterion(preds, cal)
            val_losses.append(loss.item())
            val_mae.append(torch.abs(preds - cal).mean().item())

            all_preds.extend(preds.cpu().numpy())
            all_true.extend(cal.cpu().numpy())

    avg_val_loss = np.mean(val_losses)
    avg_val_mae = np.mean(val_mae)
    val_rmse = np.sqrt(np.mean((np.array(all_preds) - np.array(all_true)) ** 2))

    print(f"[ResNet50-RGB] Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {avg_train_loss:.2f} | Train MAE: {avg_train_mae:.2f} kcal || "
          f"Val RMSE: {val_rmse:.2f} kcal | Val MAE: {avg_val_mae:.2f} kcal")

    # ---- Save Best Model ----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({'model_state_dict': model_rgb.state_dict()},
                   checkpoint_path) # Save to the updated checkpoint_path
        print(f"✅ Saved Best RGB ResNet-50 Model to {os.path.basename(checkpoint_path)}")

    scheduler.step()

print("✅ RGB-only ResNet50 Training Complete!")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 171MB/s]


❌ Checkpoint could not be loaded. Starting RGB-only ResNet-50 training from scratch. Saving to best_resnet50_rgb_retrained.pt


[ResNet50-RGB] Epoch 1 Training:   0%|          | 0/320 [00:00<?, ?it/s]

[ResNet50-RGB] Epoch 1 Validation:   0%|          | 0/40 [00:00<?, ?it/s]

...

[ResNet50-RGB] Epoch 49 Validation:   0%|          | 0/40 [00:00<?, ?it/s]

[ResNet50-RGB] Epoch 49/50 | Train Loss: 22.10 | Train MAE: 26.52 kcal || Val RMSE: 61.51 kcal | Val MAE: 39.74 kcal


[ResNet50-RGB] Epoch 50 Training:   0%|          | 0/320 [00:00<?, ?it/s]

[ResNet50-RGB] Epoch 50 Validation:   0%|          | 0/40 [00:00<?, ?it/s]

[ResNet50-RGB] Epoch 50/50 | Train Loss: 21.13 | Train MAE: 25.50 kcal || Val RMSE: 64.98 kcal | Val MAE: 41.67 kcal
✅ RGB-only ResNet50 Training Complete!


In [ ]:
# --- Final evaluation ---
model_rgb.eval()
test_losses, test_mae = [], []

with torch.no_grad():
    for rgb, cal in tqdm(test_loader, desc="[ResNet50-RGB] Final Testing"):
        rgb, cal = rgb.to(device), cal.to(device)
        preds = model_rgb(rgb).squeeze()
        loss = criterion(preds, cal)
        test_losses.append(loss.item())
        test_mae.append(torch.abs(preds - cal).mean().item())

avg_test_loss = np.mean(test_losses)
avg_test_mae = np.mean(test_mae)
test_rmse = np.sqrt(avg_test_loss)

print(f"📊 Final Test Results | RMSE: {test_rmse:.2f} kcal | MAE: {avg_test_mae:.2f} kcal")

In [48]:
import streamlit as st
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import io
import pickle
import numpy as np
import torchvision.models as models

st.set_page_config(page_title="SnapCal", layout="centered", initial_sidebar_state="auto")
with open('app.py', 'w') as f:
  f.write('''
import streamlit as st
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import io
import pickle
import numpy as np
import os
from torchvision.transforms import Grayscale
import cv2 # Keep cv2 import for image processing
import torchvision.models as models # Import torchvision models

# ---- Stylish Hero Section with Background ----

background_img_url = "https://images.pexels.com/photos/70497/pexels-photo-70497.jpeg?auto=compress&fit=crop&w=1350&q=80"
icon_url = "https://cdn-icons-png.flaticon.com/512/1046/1046857.png" # food plate icon

st.set_page_config(page_title="SnapCal", layout="centered", initial_sidebar_state="auto")

st.markdown(
     f\"\"\"
    <style>
    body {{
        background: linear-gradient(120deg, #FCFF6C 0%, #f3f4f9 100%);
        font-family: 'Segoe UI', 'Roboto', sans-serif;
    }}
    .stApp {{
        background-image: url('{background_img_url}');
        background-size: cover;
        background-attachment: fixed;
    }}
    .hero-card {{
        background: rgba(255,255,255,0.80);
        border-radius: 32px;
        padding: 2.5rem 2rem 2.5rem 2rem;
        margin: 2rem auto 2rem auto;
        box-shadow: 0 7px 32px 4px rgba(16,32,45,0.10);
        max-width: 600px;
        position: relative;
        backdrop-filter: blur(2.5px);
    }}
    .snapcal-title {{
        font-family: 'Montserrat', 'Segoe UI', Arial, sans-serif;
        font-weight: 900;
        font-size: 3rem;
        color: #1e3a8a;
        margin-bottom: .25rem;
        letter-spacing: 2.5px;
        text-shadow: 0 2px 16px #fefbde, 0 1px 4px #ffe5b1;
        text-align: center;
    }}
    .app-description {{
        color: #3d4f77;
        font-size: 1.13rem;
        text-align: center;
        margin-bottom: .5rem;
        line-height: 1.7;
    }}
    .snapcal-divider {{
        margin: 2.5rem 0 2rem 0; height:2px; background:#ffdc51; border-radius:20px;
        border:none;
    }}
    .title-icon {{
        width:55px; display:block; margin: 0 auto 6px auto; filter: drop-shadow(0 2px 6px #fff9c25e);
    }}
    .stButton>button {{
        background: linear-gradient(90deg,#ffdc51,#f8ffae 90%);
        color: #1e3a8a;
        font-weight: 700;
        border-radius: 10px;
        border: none;
        font-size: 1.17rem;
        margin-top: .5rem;
        padding: 12px 28px;
        transition: background .2s, color .2s;
        box-shadow:0 2px 8px #efefdb42;
    }}
    .stButton>button:hover {{
        background: #ffe03c;
        color: #007bff;
    }}
    .stFileUploader > div[data-testid="stFileUploaderDropzone"] {{
        border: 2px dashed #1e3a8a;
        border-radius: 12px;
        background: #fffbe7bb;
        color: #1e3a8a;
    }}
    .stFileUploader label {{
        font-size: 1.04rem;
        color: #18306e;
        letter-spacing: 0.5px;
    }}
    .stAlert {{
        border-radius: 10px;
    }}
    </style>
  \"\"\",
    unsafe_allow_html=True
)

st.markdown(
    f\"\"\"
    <div class="hero-card">
        <img src="{icon_url}" class="title-icon" alt="SnapCal Logo">
        <div class="snapcal-title">SnapCal</div>
        <div class="app-description">
            AI-powered calorie estimation from a single photo.<br>
            Upload a meal image, and SnapCal will instantly analyze and predict its calories.<br>
            <span style='color:#FFD700;font-weight:600;'>Eat smart, snap fast, stay healthy.</span>
        </div>
    </div>
    \"\"\",
    unsafe_allow_html=True
)

# ---- Main App Logic (Buttons, Upload, Results) ----

st.write("")
st.markdown('<hr class="snapcal-divider"/>', unsafe_allow_html=True)

if 'show_camera' not in st.session_state:
    st.session_state.show_camera = False

if st.button("Take a Photo"):
    st.session_state.show_camera = True

uploaded_file = st.file_uploader(
    "Upload your meal photo (.png, .jpg, .jpeg)...",
    type=["png", "jpg", "jpeg"],
    key="main-uploader"
)

camera_image = st.camera_input("Or capture with webcam") if st.session_state.show_camera else None

image = None
if uploaded_file is not None:
    from PIL import Image
    image = Image.open(uploaded_file).convert("RGB")
elif camera_image is not None:
    from PIL import Image
    image = Image.open(camera_image).convert("RGB")

if image is not None:
    st.image(image, caption="Your submitted meal", use_column_width=True)
    if st.button("Estimate Calories"):
        # --- Model inference placeholder ---
        # Replace this with your PyTorch/TorchVision prediction code!
        import random
        est_kcal = random.uniform(230, 600)
        st.success(f"Estimated Calories: {est_kcal:.1f} kcal")
        st.info("Nutritional estimate based on AI analysis. For best results, use clear and well-lit meal images.")

else:
    st.write(
        '<div style="text-align:center;color:#888;font-size:1.1rem;">'
        'No image uploaded yet.<br>Drag and drop, browse, or use the camera above.'
        '</div>',
        unsafe_allow_html=True
    )

st.write("")
st.markdown('<hr class="snapcal-divider"/>', unsafe_allow_html=True)

st.caption("© 2025 SnapCal – Creative Technologies Capstone")
''')

2025-11-11 00:21:43.942 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [49]:
#Initialize Front end with port and link

# Stop any potentially lingering streamlit processes more aggressively
!kill -9 $(lsof -i :8501 -t) >/dev/null 2>&1 || true
!kill -9 $(lsof -i :8502 -t) >/dev/null 2>&1 || true
!pkill -f streamlit
print("Attempted to kill all streamlit processes.")

# Add a small delay to ensure processes are stopped
import time
time.sleep(7) # Increased sleep time

# Install necessary libraries before running the app
!pip install pyngrok streamlit opencv-python

# Add another delay after installation
time.sleep(10)


# Restart the streamlit app on a new port
!streamlit run app.py --server.port 8503 > streamlit.log 2>&1 &

from pyngrok import ngrok
import os
from google.colab import userdata
import time

# Get your authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
if NGROK_AUTH_TOKEN is None:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    print("Waiting for Streamlit app to start...")
    # Increased delay to give Streamlit more time to start and ngrok to connect
    time.sleep(15)

    try:
        # Kill existing ngrok tunnels before connecting
        ngrok.kill()
        print("Killed existing ngrok tunnels before starting a new one.")
        # Connect to the new port 8503
        public_url = ngrok.connect(8503)
        print(f"Streamlit app URL: {public_url}")
    except Exception as e:
        print(f"Error connecting ngrok: {e}")
        print("Please check if Streamlit is running and accessible on port 8503.")

Attempted to kill all streamlit processes.


Waiting for Streamlit app to start...
Killed existing ngrok tunnels before starting a new one.
Streamlit app URL: NgrokTunnel: "https://incommunicative-funnily-melda.ngrok-free.dev" -> "http://localhost:8503"
